# Computing Reduced Density Matrices from MPO

### Basics Imports

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    #level=logging.INFO
    level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np
import pandas as pd
import sys

### TensorNetworks imports

In [ ]:
sys.path.append("../")
import tensornetworks as tn
import tn_quantum_circuits as tnqc
import gates as gt
from tensornetworks import contract_indices, contract_indices_one_tensor

### Imports QLM

In [ ]:
import qat.lang.AQASM as qlm
from qat.qpus import PyLinalg
qpu_p = PyLinalg()
from qlm_stuff import proccess_qresults

## 1. Functions for creating the MPS

In [ ]:
def apply_2qubit_gates(qubits, gates):
    """
    Executes product of tensor with a gate
    -o-o-o-o-o-..o-o-
     |   |   |     |
    """
    new_qubits = [0 for i in qubits]
    left = qubits[0]
    for i in range(1, len(qubits)):
        right = qubits[i]
        gate = gates[i-1]
        #new_qubits[i-1], left = phase_change(left, right, gate)
        new_qubits[i-1], left = tnqc.apply_2qubit_gate(left, right, gate)

    new_qubits[-1], new_qubits[0] = tnqc.apply_2qubit_gate(
        left, new_qubits[0], gates[-1])
     #new_qubits[-1], new_qubits[0] = phase_change(left, new_qubits[0], gates[-1])
    return new_qubits

In [ ]:
def get_angles(depth):
    theta = np.pi/4.0
    delta_theta = theta / (depth + 1)
    angles = []
    for i in range(depth):
        angles.append([(2 * i + 1) * delta_theta, (2 * i + 2) * delta_theta])
    return angles     

In [ ]:
def ansatz(nqubits, depth, angles):
    # Intitial State
    zeroket = np.zeros((1, 2, 1))
    zeroket[0][0][0] = 1
    zeroket = zeroket.astype(complex)
    #Initial State
    mps_ = [zeroket] * nqubits
    for depth_ in range(depth):
        # First Layer
        gates = [gt.x_rotation(angles[depth_][0]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
        ent_gates = [gt.controlz() for i in mps_]
        mps_ = apply_2qubit_gates(mps_, ent_gates)
        gates = [gt.z_rotation(angles[depth_][1]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
    return mps_

In [ ]:
def ansatz_qlm(nqubits, depth, angles):
    qprog = qlm.Program()
    qbits = qprog.qalloc(nqubits)
    for d_ in range(0, depth):
        for i in range(nqubits):
            qprog.apply(qlm.RX(angles[d_][0]), qbits[i])
        for i in range(nqubits-1):
            qprog.apply(qlm.Z.ctrl(), qbits[i], qbits[i+1])    
        qprog.apply(qlm.Z.ctrl(), qbits[nqubits-1], qbits[0])
        for i in range(nqubits):
            qprog.apply(qlm.RZ(angles[d_][1]), qbits[i])    
    circ = qprog.to_circ()
    #%qatdisplay circ
    job = circ.to_job()
    state = qpu_p.submit(job)
    pdf = proccess_qresults(state, nqubits)
    pdf.reset_index(drop=True, inplace=True)
    return pdf, circ  

## 2. Creating MPS

In [ ]:
# MPS uisng My code
depth = 3
nqubits = 10

In [ ]:
free = [0, 1, 2, 3, 4, 5, 6, 7]
contraction = list(range(nqubits))
contraction = [i for i in contraction if i not in free]

In [ ]:
%%time
mps = ansatz(nqubits, depth, get_angles(depth))
#rho_mps = reduced_rho_mps(mps, free, contraction)
#rho_mps_z = reduced_rho_mpo_z(mps, free, contraction)

In [ ]:
%%time
#Stat of circuit for comparing with MPS computations
pdf, c= ansatz_qlm(nqubits, depth,  get_angles(depth))
# Computing Density Matrix using reduced_matrix for pure tensors
amp = np.array(pdf["Amplitude"])
amp = amp.reshape(tuple([2 for i in range(nqubits)]))
rho_qlm = tn.reduced_matrix(amp, free, contraction)

## 3. Reduced Rho

In [ ]:
from tensornetworks import mpo_contraction
def contraction_pl(tensor):
    tensor = contract_indices(tensor, tensor.conj(), [1], [1])
    tensor = tensor.transpose(0, 2, 1, 3)
    reshape = [
        tensor.shape[0] * tensor.shape[1], 
        tensor.shape[2] * tensor.shape[3]
    ]
    tensor = tensor.reshape(reshape)
    return tensor

def reduced_rho_mps(mps, free_indices, contraction_indices):
    # First deal with contraction indices
    tensor_contracted = contraction_pl(mps[contraction_indices[0]])
    print(tensor_contracted.shape)
    for i in contraction_indices[1:]:
        #print(i)
        tensor = contraction_pl(mps[i])
        tensor_contracted = mpo_contraction(tensor_contracted, tensor)    
    # Second deal with free indices
    tensor_free = mps[free_indices[0]]
    print(tensor_free.shape)
    for i in free_indices[1:]:
        #print(i)
        tensor = mps[i]
        #print(tensor.shape)
        tensor_free= contract_indices(tensor_free, tensor, [2], [0])
        #print(tensor_free.shape)
        reshape = [
            tensor_free.shape[0] , 
            tensor_free.shape[1] * tensor_free.shape[2],
            tensor_free.shape[3], 
        ]
        tensor_free = tensor_free.reshape(reshape)  
    
    tensor_free = contract_indices(tensor_free, tensor_free.conj(), [], [])
    tensor_free = tensor_free.transpose(0, 3, 1, 4, 2, 5)
    reshape = [
        tensor_free.shape[0] * tensor_free.shape[1],
        tensor_free.shape[2],  tensor_free.shape[3],
        tensor_free.shape[4] * tensor_free.shape[5]
    ]
    tensor_free = tensor_free.reshape(reshape)
    print(tensor_free.shape, tensor_contracted.shape)
    tensor_out = contract_indices(tensor_free, tensor_contracted, [3, 0], [0, 1])
    return tensor_out

In [ ]:
rho_mps = reduced_rho_mps(mps, free, contraction)

In [ ]:
np.isclose(rho_mps, rho_qlm).all()